In [1]:
import sys
import os
import pandas as pd 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

from recommenders.datasets.amazon_reviews import get_review_data
from recommenders.datasets.split_utils import filter_k_core
from recommenders.models.sasrec.model import SASREC
from recommenders.models.sasrec.ssept import SSEPT
from recommenders.models.sasrec.sampler import WarpSampler
from recommenders.models.sasrec.util import SASRecDataSet
from recommenders.utils.notebook_utils import store_metadata
from recommenders.utils.timer import Timer


print(f"System version: {sys.version}")
print(f"Tensorflow version: {tf.__version__}")

System version: 3.9.19 (main, Mar 21 2024, 17:21:27) [MSC v.1916 64 bit (AMD64)]
Tensorflow version: 2.12.0


In [2]:
data_dir = os.path.join("./")

# SerenLens modified
dataset = "reviews_Books_5_full"

In [3]:
outfile = dataset + '.txt'

In [9]:
df = pd.read_csv("../Data/SerenLens_Books.csv")
df.head()

,user_id,item_id,timestamp,review,rating,label
0,a10e3f50diujee,0061148512,1196899200,i have always loved this book. plath's litera...,5.0,0.0
1,a10e3f50diujee,0786718617,1199750400,assia is infamous for being the woman who ende...,4.0,0.0
2,a10e3f50diujee,0446617687,1212624000,"because i am a fan of the show, i sought out t...",3.0,0.0
3,a10e3f50diujee,0375725601,1212969600,erik larson took on quite a task with this wor...,5.0,0.0
4,a10e3f50diujee,0141800356,1215043200,"although there are hundreds of reviews, i had ...",5.0,0.0


In [10]:
df['userID'] = df['user_id'].str.lower()
df['itemID'] = df['item_id'].str.lower()
df['time'] = df['timestamp']

df=df[['userID','itemID', 'time', 'label']]

df.head()

,userID,itemID,time,label
0,a10e3f50diujee,0061148512,1196899200,0.0
1,a10e3f50diujee,0786718617,1199750400,0.0
2,a10e3f50diujee,0446617687,1212624000,0.0
3,a10e3f50diujee,0375725601,1212969600,0.0
4,a10e3f50diujee,0141800356,1215043200,0.0


In [11]:
user_set, item_set = set(df['userID'].unique()), set(df['itemID'].unique())
user_map = dict()
item_map = dict()
for u, user in enumerate(user_set):
    user_map[user] = u+1
for i, item in enumerate(item_set):
    item_map[item] = i+1

df["userID"] = df["userID"].apply(lambda x: user_map[x])
df["itemID"] = df["itemID"].apply(lambda x: item_map[x])
df = df.sort_values(by=["userID", "time"])
df.drop(columns=["time"], inplace=True)

In [12]:
count_label_1_seren = df[df['label'] == 1.0].shape[0]


# Exibindo o resultado
print("Número de entradas com label igual a 1.0 seren:", count_label_1_seren)

Número de entradas com label igual a 1.0 seren: 2557


In [13]:
df.to_csv(os.path.join(data_dir, outfile), sep="\t", header=False, index=False)